### Ref1: https://rateslib.readthedocs.io/en/stable/z_swpm.html
### Ref2: https://quant.stackexchange.com/a/49585

### Default SOFR

![Default SOFR](dv01/default_sofr.png)

### Swap

![Swap](dv01/swap.png)

In [1]:
import pandas as pd
from rateslib import *

In [2]:
def get_pv_results(bump=0.0):
    data = pd.DataFrame({
    "Term": ["1W", "2W", "3W", "1M", "2M", "3M", "4M", "5M", "6M", "7M", "8M", "9M", "10M", "11M", "12M", "18M", "2Y", "3Y", "4Y"],
    "Rate": [5.30111, 5.30424, 5.30657, 5.31100, 5.34800, 5.38025, 5.40915, 5.43078, 5.44235, 5.44950, 5.44878, 5.44100, 5.42730, 5.40747, 5.3839, 5.09195, 4.85785, 4.51845, 4.31705],
    })
    
    data["Rate"] += bump

    
    data["Termination"] = [add_tenor(dt(2023, 8, 21), _, "F", "nyc") for _ in data["Term"]]
    
    sofr = Curve(
        id="sofr",
        convention="Act360",
        calendar="nyc",
        modifier="MF",
        interpolation="log_linear",
        nodes={
            **{dt(2023, 8, 17): 1.0},  # <- this is today's DF,
            **{_: 1.0 for _ in data["Termination"]},
        }
    )

    sofr_args = dict(effective=dt(2023, 8, 21), spec="usd_irs", curves="sofr")

    solver = Solver(
        curves=[sofr],
        instruments=[IRS(termination=_, **sofr_args) for _ in data["Termination"]],
        s=data["Rate"],
        instrument_labels=data["Term"],
        id="us_rates",
    )

    data["DF"] = [float(sofr[_]) for _ in data["Termination"]]
    
    irs = IRS(
        effective=dt(2023, 11, 21),
        termination=dt(2025, 2, 21),
        notional=-100e6,
        fixed_rate=5.40,
        curves="sofr",
        spec="usd_irs",
    )
    
    npv = irs.npv(solver=solver)
    dv01 = irs.delta(solver=solver).sum()
    return npv, dv01

In [3]:
npv, dv01 = get_pv_results(bump=0.0)

SUCCESS: `func_tol` reached after 5 iterations (levenberg_marquardt) , `f_val`: 3.116743265440467e-17, `time`: 0.0750s


### PV

In [4]:
print("PV", npv.real)

PV 456622.09860395174


### DV01

In [5]:
print("DV01", dv01)

DV01 local_ccy  display_ccy
usd        usd           -11879.936805
dtype: float64


### DV01 - Central Finite Difference

Assume the PnL on a swap is almost its linear pnl plus its convexity:

$\Delta P (\Delta r) \approx \frac{\partial P}{\partial r} \Delta r  + \frac{1}{2} \frac{{\partial}^2 P}{\partial r^2} \Delta r^2$

Then bumping by +1bp and -1bp, dividing by 2 eliminates the convexity element and very accurately approximates the real PV01:

$\frac{\Delta P (+1bp) - \Delta P (-1bp)}{2} = \frac{\partial P}{\partial r}$

In [12]:
one_bp = 0.01

In [13]:
npv_plus_1bp, _ = get_pv_results(bump=one_bp)
npv_plus_1bp_val = npv_plus_1bp.real

SUCCESS: `func_tol` reached after 5 iterations (levenberg_marquardt) , `f_val`: 3.108013716766087e-17, `time`: 0.0754s


In [14]:
npv_minus_1bp, _ = get_pv_results(bump=-one_bp)
npv_minus_1bp_val = npv_minus_1bp.real

SUCCESS: `func_tol` reached after 5 iterations (levenberg_marquardt) , `f_val`: 3.1254963583773556e-17, `time`: 0.0770s


In [15]:
dv01_finite_diff = (npv_plus_1bp_val - npv_minus_1bp_val) / 2.0
print(dv01_finite_diff)

-11879.93697656691


### DV01 - Single Bumped Curve
Another common method of calculation is to use a single bumped curve by, say, $\frac{1}{100}$ th of a bp, and scale the result by 100. Although less accurate, since the convexity is marginalised and not eliminated, the calculation is twice as fast, for example:

$100 \Delta P (+\frac{1}{100}bp) = \frac{\partial P}{\partial r} + \frac{1}{200} \frac{{\partial}^2 P}{\partial r^2} $

In [16]:
npv_bump, _ = get_pv_results(bump=one_bp*0.01)
npv_bump_val = npv_bump.real

SUCCESS: `func_tol` reached after 5 iterations (levenberg_marquardt) , `f_val`: 3.116670055661461e-17, `time`: 0.0765s


In [17]:
dv01_single_bump = (npv.real - npv_bump_val) * 100.0
print(dv01_single_bump)

11879.920919146389
